In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from gtts import gTTS
from mtranslate import translate
import pyperclip
import os
from langdetect import detect
import pygame

# Initialize Tkinter window
root = tk.Tk()
root.geometry("800x600")
root.title("Language Translation")

# Apply a theme for better styling
style = ttk.Style()
style.theme_use('clam')

# Read language dataset
df = pd.read_csv(r"D:\ABCDEFG\N_Batch -- 5.30PM\4. JULY\22nd- Twitter sentiment analysis\MULTIPLE LANGUAGE TRANSLATION\MULTIPLE LANGUAGE TRANSLATION\language.csv")
df.dropna(inplace=True)
lang = df['name'].to_list()
langlist = tuple(lang)
langcode = df['iso'].to_list()

# Create dictionary of language and 2 letter langcode
lang_array = {lang[i]: langcode[i] for i in range(len(langcode))}

# Layout
root.configure(bg='#E8F0FE')  # Set background color
tk.Label(root, text="Language Translation", font="Helvetica 24 bold", bg='#E8F0FE').pack(pady=20)

frame = tk.Frame(root, bg='#E8F0FE')
frame.pack(pady=10)

# Input text box
inputtext = tk.Text(frame, height=5, width=50, font='Helvetica 12')
inputtext.pack(pady=10)

# Language selection dropdown
choice = tk.StringVar()
choice.set(langlist[0])
ttk.OptionMenu(frame, choice, *langlist).pack(pady=10)

# Output text box
outputtext = tk.Text(frame, height=5, width=50, font='Helvetica 12', bg='#F5F5F5')
outputtext.pack(pady=10)

# Function to translate text
def translate_text():
    try:
        input_text = inputtext.get("1.0", tk.END).strip()
        if input_text:
            translated_text = translate(input_text, lang_array[choice.get()])
            outputtext.delete("1.0", tk.END)
            outputtext.insert(tk.END, translated_text)
        else:
            messagebox.showwarning("Input Error", "Please enter text to translate.")
    except Exception as e:
        messagebox.showerror("Translation Error", str(e))

# Function to copy translated text
def copy_text():
    output_text = outputtext.get("1.0", tk.END).strip()
    pyperclip.copy(output_text)
    messagebox.showinfo("Copy Success", "Translated text copied to clipboard!")

# Function to save audio file of translated text
def save_audio():
    try:
        output_text = outputtext.get("1.0", tk.END).strip()
        if output_text:
            tts = gTTS(text=output_text, lang=lang_array[choice.get()], slow=False)
            file_path = filedialog.asksaveasfilename(defaultextension=".mp3", filetypes=[("MP3 files", "*.mp3")])
            if file_path:
                tts.save(file_path)
                messagebox.showinfo("Save Success", f"Audio file saved as {file_path}")
        else:
            messagebox.showwarning("Output Error", "No translated text to save.")
    except Exception as e:
        messagebox.showerror("Audio Save Error", str(e))

# Function to clear input and output text boxes
def clear_text():
    inputtext.delete("1.0", tk.END)
    outputtext.delete("1.0", tk.END)

# Function to speak the translated text
def speak_translation_text():
    try:
        output_text = outputtext.get("1.0", tk.END).strip()
        if output_text:
            tts = gTTS(text=output_text, lang=lang_array[choice.get()], slow=False)
            tts.save("temp.mp3")
            pygame.mixer.init()
            pygame.mixer.music.load("temp.mp3")
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                continue
            os.remove("temp.mp3")
        else:
            messagebox.showwarning("Output Error", "No translated text to speak.")
    except Exception as e:
        messagebox.showerror("Speak Text Error", str(e))

# Function to provide text statistics
def text_statistics():
    try:
        input_text = inputtext.get("1.0", tk.END).strip()
        if input_text:
            word_count = len(input_text.split())
            char_count = len(input_text)
            stats_message = f"Word Count: {word_count}\nCharacter Count: {char_count}"
            messagebox.showinfo("Text Statistics", stats_message)
        else:
            messagebox.showwarning("Input Error", "Please enter text to get its statistics.")
    except Exception as e:
        messagebox.showerror("Text Statistics Error", str(e))

# Function to summarize the text
def summarize_text():
    try:
        input_text = inputtext.get("1.0", tk.END).strip()
        if input_text:
            summarizer = pipeline("summarization")
            summarized_text = summarizer(input_text, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
            outputtext.delete("1.0", tk.END)
            outputtext.insert(tk.END, summarized_text)
        else:
            messagebox.showwarning("Input Error", "Please enter text to summarize.")
    except Exception as e:
        messagebox.showerror("Summarization Error", str(e))

# Function to translate text into multiple languages
def multiple_translation():
    try:
        input_text = inputtext.get("1.0", tk.END).strip()
        if input_text:
            translations = {lang: translate(input_text, lang_array[lang]) for lang in langlist}
            outputtext.delete("1.0", tk.END)
            for language, translation in translations.items():
                outputtext.insert(tk.END, f"{language}:\n{translation}\n\n")
        else:
            messagebox.showwarning("Input Error", "Please enter text to translate.")
    except Exception as e:
        messagebox.showerror("Translation Error", str(e))

# Function to exit the application
def exit_app():
    root.quit()

# Add button animations and text animations
def button_hover(event):
    event.widget['background'] = '#76C7C0'

def button_leave(event):
    event.widget['background'] = '#4CAF50'

def text_animation():
    inputtext.tag_configure('blink', background='yellow')
    current_tags = inputtext.tag_names('insert')
    if 'blink' in current_tags:
        inputtext.tag_remove('blink', 'insert')
    else:
        inputtext.tag_add('blink', 'insert')
    root.after(500, text_animation)

# Buttons
button_frame = tk.Frame(frame, bg='#E8F0FE')
button_frame.pack(pady=20)

# Define button styles
style.configure('TButton', font='Helvetica 12', padding=6)
style.configure('TButton', background='#4CAF50', foreground='white')

buttons = [
    ttk.Button(button_frame, text="Translate", command=translate_text, style='TButton'),
    ttk.Button(button_frame, text="Copy", command=copy_text, style='TButton'),
    ttk.Button(button_frame, text="Save Audio", command=save_audio, style='TButton'),
    ttk.Button(button_frame, text="Clear", command=clear_text, style='TButton'),
    ttk.Button(button_frame, text="Speak Translation Text", command=speak_translation_text, style='TButton'),
    ttk.Button(button_frame, text="Summarize Text", command=summarize_text, style='TButton'),
    ttk.Button(button_frame, text="Multiple Translation", command=multiple_translation, style='TButton'),
    ttk.Button(button_frame, text="Text Statistics", command=text_statistics, style='TButton'),
    ttk.Button(button_frame, text="Exit", command=exit_app, style='TButton')
]

for i, button in enumerate(buttons):
    button.grid(row=i//3, column=i%3, padx=10, pady=10)
    button.bind("<Enter>", button_hover)
    button.bind("<Leave>", button_leave)

# Start text animation
text_animation()

root.mainloop()
